In [ ]:
### First time data download, adding moving averages, saving to DB

import yfinance as yf
import pandas as pd
import sqlite3

stock_symbol = "^NDX"

# Define the date range for the last 2 months
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(months=1)

# Download data using yfinance
nasdaq_data = yf.download(stock_symbol, start=start_date, end=end_date)

#SMAs

nasdaq_data['SMA40'] = nasdaq_data['Close'].rolling(40).mean()
nasdaq_data['SMA85'] = nasdaq_data['Close'].rolling(85).mean()

#optional - drop NAs- data.dropna(inplace=True)

# Save the DataFrame to the database - using 'replace' for the first time only

conn = sqlite3.connect('nasdaq_D.db')
nasdaq_data.to_sql('nasdaq_D', conn, if_exists='replace')

#import data from DB
nasdaq_data = pd.read_sql_query("SELECT * FROM nasdaq_D", conn, parse_dates=['Date'])
nasdaq_data.set_index('Date')

#removing the last row
nasdaq_data.drop(nasdaq_data.index[len(nasdaq_data)-1], inplace=True)

conn.close()

In [ ]:
# Adding new prices, appending to the existing DB, alerting the result
# to be debugged

import yfinance as yf
import pandas as pd
import sqlite3
import win32api

conn = sqlite3.connect('nasdaq_D.db')

nasdaq_data = pd.read_sql_query("SELECT * FROM nasdaq_D", conn, parse_dates=['Date'])
nasdaq_data.set_index('Date')
last_date_obj = nasdaq_data.loc[len(nasdaq_data)-1]['Date']
start_date = pd.Timestamp(last_date_obj)
end_date = pd.Timestamp.now()

# Define the stock symbol (e.g., for NASDAQ)
stock_symbol = "^NDX"

# function that adds moving averages and RSI

def add_MAs():
    nq = pd.read_sql_query("SELECT * FROM nasdaq_D", conn, parse_dates=['Date'])
    nq.set_index('Date')
    if nq.at[nq.index[-1],'SMA40'] == 0.0:
        nq['SMA40'] = (nq['Close']).rolling(40).mean()
        nq['SMA85'] = (nq['Close']).rolling(85).mean()
        # Calculate the price change for each period
        delta = nq['Close'].diff()

        # Define the period for the rolling average
        period = 14

        # Calculate the average gain and average loss for the specified period
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(period).mean()
        avg_loss = loss.rolling(period).mean()

        # Calculate the Relative Strength (RS) by dividing the average gain by the average loss
        rs = avg_gain / avg_loss

        # Calculate the Relative Strength Index (RSI)
        rsi = 100 - (100 / (1 + rs))

        # Add the RSI to the dataframe
        nq['RSI'] = rsi
        
        return 1
    else:
        return 0
    
# main code

if end_date.day > (end_date-pd.DateOffset(days=1)).day:
    data = yf.download(stock_symbol, start=start_date, end=end_date)
    data.reset_index(inplace=True)

    if data.iloc[0]['Date'] > start_date:

        
        data['SMA40'] = 0.0
        data['SMA85'] = 0.0
        data['RSI'] = 0.0

        # Save the DataFrame to the database
        data.to_sql('nasdaq_D', conn, if_exists='append')
        result = add_MAs()
        win32api.MessageBox(0, f'US100 1d data updation result: {result}', 'Price Updater', 0x00001000) 
    
    else: 
        win32api.MessageBox(0, 'No new price data for US100', 'Price Updater', 0x00001000) 
    
else: 
    win32api.MessageBox(0, 'No new price data for US100', 'Price Updater', 0x00001000) 
conn.close()


In [ ]:
# MAs cross identification
import pandas as pd
import sqlite3
import numpy as np

conn = sqlite3.connect('nasdaq_D.db')
nasdaq_data = pd.read_sql_query("SELECT * FROM nasdaq_D", conn, parse_dates=['Date'])
MA_crosses = []

for i in range(len(nasdaq_data)):
    today40 = float(nasdaq_data.iloc[i]['SMA40'])
    today85 = float(nasdaq_data.iloc[i]['SMA85'])
    try:
        yesterday40 = float(nasdaq_data.iloc[i-1]['SMA40'])
        yesterday85 = float(nasdaq_data.iloc[i-1]['SMA85'])
        
        
        if today40 > today85:
            if yesterday85 > yesterday40:
                MA_crosses.append({'index':i,'side':'L','one_month+top%':0.0,'one_month-low%':0.0})
                
        if today40 < today85:
            if yesterday85 < yesterday40:
                MA_crosses.append({'index':i,'side':'S','one_month+top%':0.0,'one_month-low%':0.0})
        
        
    except:
        pass
    
#function calculating the distance between close and MA
    
def calc_odl(ma,close):
    dif = ma / close
    odl = ''
    if dif < 1:
        odl = 1 - dif
    elif dif > 1:
        odl = dif - 1
        print(odl)
    return odl       

#adding the highest monthly price from signal %-wise, and lowest
for i in range(len(MA_crosses)):
    ind = MA_crosses[i]['index']
    op = float(nasdaq_data.iloc[int(ind)+1]['Open'])
    highest_price = 0.0
    lowest_price = 100000.00
    hperc = ''
    lperc = ''
    try:
        for r in range(30):
            close1 = float(nasdaq_data.iloc[int(ind)+r]['Close'])
            if close1 > highest_price:
                highest_price = close1
                hperc = highest_price / op
                
            if close1 < lowest_price:
                lowest_price = close1
                lperc = lowest_price / op
    except:
        pass
    if hperc != '' and lperc != '':
        MA_crosses[i]['one_month+top%'] = hperc
        MA_crosses[i]['one_month-low%'] = lperc
        tp = (hperc - 1)*1000
        sl = (lperc - 1)*1000
        MA_crosses[i]['RR'] = abs(tp/sl)
        sma85 = float(nasdaq_data.iloc[ind]['SMA85'])
        print(sma85)
        clos = float(nasdaq_data.iloc[ind]['Close'])
        odl_85_od_close = calc_odl(sma85,clos)
        MA_crosses[i]['odl_od_85'] = odl_85_od_close
        

# Save the DataFrame to the database
nasdaq_data.to_sql('nasdaq_D', conn, if_exists='append')
conn.close()